In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
githubPath = 'https://raw.githubusercontent.com/clicksign/python-dev-test/master/data/Adult.test'
db_name = '/home/god/git/us_census_bureau.db'
qtd_linhas_por_carga = 1630

In [3]:
try:
    conn = sqlite3.connect(db_name)
    qtd = pd.read_sql_query("select max(index_col) qtd from df", conn)
except Exception as e:
    print(e)
    cmd = 'rm -rf {}'.format(db_name)
    print(cmd)
    os.system(cmd)
    cmd = 'touch {}'.format(db_name)
    print(cmd)
    os.system(cmd)
    conn = sqlite3.connect(db_name)
    qtd = pd.DataFrame({'qtd': [0]})

Execution failed on sql 'select max(index_col) qtd from df': no such table: df
rm -rf /home/raft/git/us_census_bureau.db
touch /home/raft/git/us_census_bureau.db


In [4]:
df = pd.read_csv(githubPath,
                 names=[
                     'age', 'workclass', 'fnlwgt', 'education',
                     'education-num', 'marital-status', 'occupation',
                     'relationship', 'race', 'sex', 'capital-gain',
                     'capital-loss', 'hours-per-week', 'native-country',
                     'class'
                 ])

In [5]:
df[df.index == 3480]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
3480,64,?,226878.0,Masters,14.0,Married-civ-spouse,?,Wife,Black,Female,9386.0,0.0,50,Jamaica,>50K.


In [6]:
df['index_col'] = df.index

In [7]:
ini = int(qtd['qtd'][0]) + 1
fim = ini + qtd_linhas_por_carga
df = df[ini:fim]
print(ini, fim)

1 1631


In [8]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,index_col
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40,United-States,<=50K.,1
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50,United-States,<=50K.,2
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40,United-States,>50K.,3
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40,United-States,>50K.,4
5,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30,United-States,<=50K.,5


In [9]:
dominio = {
    'workclass': [
        'Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov',
        'Local-gov', 'State-gov', 'Without-pay', 'Never-worked'
    ],
    'education': [
        'Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school',
        'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters',
        '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool'
    ],
    'marital-status': [
        'Married-civ-spouse', 'Divorced', 'Never-married', 'Separated',
        'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'
    ],
    'occupation': [
        'Tech-support', 'Craft-repair', 'Other-service', 'Sales',
        'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners',
        'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing',
        'Transport-moving', 'Priv-house-serv', 'Protective-serv',
        'Armed-Forces'
    ],
    'relationship': [
        'Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative',
        'Unmarried'
    ],
    'race':
    ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'],
    'sex': ['Male', 'Female'],
    'native-country': [
        'United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada',
        'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece',
        'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy',
        'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland',
        'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti',
        'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland',
        'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru',
        'Hong', 'Holand-Netherlands'
    ],
    'class': ['>50K.', '<=50K.']
}

In [10]:
campos_numericos = [
    'age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

In [11]:
df = df[df.age.str.isnumeric()]

In [12]:
def trata_coluna_string(coluna, dominio):
    coluna.fillna('', inplace=True)
    coluna = coluna.apply(lambda x: x.strip())
    coluna = coluna.apply(lambda x: x.upper())
    dominio = [x.upper() for x in dominio]
    coluna = coluna[coluna.isin(dominio)]

    return coluna

In [13]:
def trata_coluna_numerico(dfc, i):
    dfc[i] = pd.to_numeric(dfc[i], errors='coerce')
    dfc[i].fillna(-2, inplace=True)
    
    return dfc[i]

In [14]:
for i, k in dominio.items():
    df[i] = trata_coluna_string(df[i], k)

In [15]:
for i in campos_numericos:
    df[i] = trata_coluna_numerico(df, i)

In [16]:
# df['hours-per-week'] = pd.to_numeric(df['hours-per-week'], errors='coerce')
# df['hours-per-week'].fillna(-2, inplace=True)

In [17]:
c = {'>50K.': True, '<=50K.': False}
df['class'] = df['class'].map(c)

In [18]:
df.fillna('não identificado', inplace=True)

In [19]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,index_col
1,25,PRIVATE,226802.0,11TH,7.0,NEVER-MARRIED,MACHINE-OP-INSPCT,OWN-CHILD,BLACK,MALE,0.0,0.0,40.0,UNITED-STATES,False,1
2,38,PRIVATE,89814.0,HS-GRAD,9.0,MARRIED-CIV-SPOUSE,FARMING-FISHING,HUSBAND,WHITE,MALE,0.0,0.0,50.0,UNITED-STATES,False,2
3,28,LOCAL-GOV,336951.0,ASSOC-ACDM,12.0,MARRIED-CIV-SPOUSE,PROTECTIVE-SERV,HUSBAND,WHITE,MALE,0.0,0.0,40.0,UNITED-STATES,True,3
4,44,PRIVATE,160323.0,SOME-COLLEGE,10.0,MARRIED-CIV-SPOUSE,MACHINE-OP-INSPCT,HUSBAND,BLACK,MALE,7688.0,0.0,40.0,UNITED-STATES,True,4
5,18,não identificado,103497.0,SOME-COLLEGE,10.0,NEVER-MARRIED,não identificado,OWN-CHILD,WHITE,FEMALE,0.0,0.0,30.0,UNITED-STATES,False,5


In [20]:
df.to_sql('df', con=conn, index=False, if_exists='append')